# Tutorial Phi-4 Mini ONNX pentru Apeluri Paralele ale Funcțiilor

Acest notebook demonstrează cum să utilizați Phi-4 Mini cu ONNX Runtime GenAI pentru apeluri paralele ale funcțiilor. Apelarea funcțiilor permite modelului să invoce inteligent instrumente și API-uri externe pe baza cererilor utilizatorului.

## Prezentare generală

În acest tutorial, veți învăța cum să:
- Configurați Phi-4 Mini cu ONNX Runtime GenAI
- Definiți scheme de funcții pentru rezervarea zborurilor și hotelurilor
- Utilizați generarea ghidată cu gramatică Lark pentru un output structurat
- Executați apeluri paralele ale funcțiilor pentru scenarii complexe de rezervare de călătorii

## Cerințe preliminare

Înainte de a rula acest notebook, asigurați-vă că:
- Ați descărcat modelul Phi-4 Mini ONNX
- Ați instalat pachetul `onnxruntime-genai`
- Aveți o înțelegere de bază a conceptelor de apelare a funcțiilor


## Pasul 1: Importarea bibliotecilor necesare

Mai întâi, vom importa bibliotecile necesare pentru implementarea apelului funcției.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Pasul 2: Configurarea și Configurația Modelului

Acum vom configura modelul Phi-4 Mini ONNX. Asigură-te că înlocuiești calea cu directorul tău actual al modelului.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Pasul 3: Configurarea Parametrilor de Generare

Configurează parametrii de generare pentru a controla comportamentul răspunsurilor modelului. Aceste setări asigură răspunsuri deterministe și concentrate pentru apelarea funcțiilor.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Pasul 4: Definirea funcțiilor disponibile

Aici definim funcțiile pe care asistentul nostru AI le poate utiliza. În acest exemplu, avem două funcții legate de călătorii:
1. **booking_flight_tickets**: Pentru rezervarea zborurilor între aeroporturi
2. **booking_hotels**: Pentru rezervarea cazărilor la hoteluri

Definițiile funcțiilor urmează formatul schema de apelare a funcțiilor de la OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Pasul 5: Funcții Ajutătoare pentru Generarea Gramaticii

Aceste funcții ajutătoare transformă definițiile funcțiilor noastre în formatul de gramatică Lark, utilizat de ONNX Runtime GenAI pentru generarea ghidată. Acest lucru asigură că modelul produce apeluri de funcții valide în formatul JSON corect.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Pasul 6: Testarea Generării Gramaticii

Să testăm funcțiile noastre de generare a gramaticii pentru a vedea cum transformă definițiile uneltelor noastre în formatul corespunzător.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Pasul 7: Pregătește Promptul Sistemului și Generatorul

Acum vom crea promptul sistemului care informează modelul despre instrumentele disponibile și vom configura generatorul cu parametrii de generare ghidată.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Pasul 8: Inițializarea Generatorului cu Generare Ghidată

Acum vom crea generatorul folosind parametrii configurați și vom aplica gramatica Lark pentru generare ghidată.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Pasul 9: Testarea Apelării Funcțiilor în Paralel

Acum să testăm configurația noastră cu un scenariu complex de rezervare de călătorii care necesită apelarea mai multor funcții.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Generarea Apelurilor de Funcții

Modelul va genera acum apeluri de funcții structurate pe baza cererii utilizatorului. Datorită generării ghidate, rezultatul va fi în format JSON valid, care poate fi executat direct.

**Rezultat Așteptat**: Modelul ar trebui să genereze apeluri de funcții pentru:
1. `booking_flight_tickets` cu detalii despre zborul Beijing (PEK) către Paris (CDG)
2. `booking_hotels` cu detalii despre cazarea în Paris

Rulați celula de mai jos pentru a vedea generarea în timp real:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Concluzie

🎉 **Felicitări!** Ai implementat cu succes apelarea funcțiilor în paralel folosind Phi-4 Mini și ONNX Runtime GenAI.

### Ce ai învățat:

1. **Configurarea modelului**: Cum să configurezi Phi-4 Mini cu ONNX Runtime GenAI
2. **Definirea funcțiilor**: Cum să definești schemele funcțiilor pentru apelarea funcțiilor AI
3. **Generare ghidată**: Cum să utilizezi gramatica Lark pentru generarea structurată a rezultatelor
4. **Apeluri de funcții în paralel**: Cum să gestionezi cereri complexe care necesită apeluri multiple de funcții

### Beneficii cheie:

- ✅ **Rezultate structurate**: Generarea ghidată asigură apeluri JSON valide ale funcțiilor
- ✅ **Procesare paralelă**: Gestionarea mai multor apeluri de funcții într-o singură cerere
- ✅ **Performanță ridicată**: ONNX Runtime oferă inferență optimizată
- ✅ **Schematizare flexibilă**: Ușor de adăugat sau modificat definițiile funcțiilor

### Resurse:

- [Documentația Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [Documentația ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)
- [Cele mai bune practici pentru apelarea funcțiilor](https://platform.openai.com/docs/guides/function-calling)



---

**Declinarea responsabilității**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși depunem eforturi pentru a asigura acuratețea, vă rugăm să aveți în vedere că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa nativă ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
